In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

from PIL import Image
import os
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-white')

In [22]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.set_default_device(device)

device

device(type='cuda')